In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df = pd.read_csv('movie_dataset.csv')

In [6]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [8]:
features = ['keywords','cast','genres','director','production_companies']

In [15]:
df1 = df[features]

making production_companies to usable row value

In [16]:
#this below function returns a string of combined production house names.
import re
v = '[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'
'''
example v's value after applying extractProduction function will
be :
'Ingenious Film Partners Twentieth Century Fox Film Corporation Dune Entertainment Lightstorm Entertainment'

getting only the prodcution company names.
'''
def extractProduction(row_value):
    #row_value is a string
    prod_house=''
    #using regular expressions to extract the name 
    rexp = "name\":.*?,"
    preprocessList = re.findall(rexp,row_value)
    rexp1 = '"(.*)"'
    for temp in preprocessList:
        temp = temp.split(":")[1]
        #using exception because some values in temp
        #is an empty list.
        try:
            house = re.findall(rexp1,temp)[0]
            prod_house+=house+" "
        except:
            pass
    return prod_house.rstrip()


df['production_companies']=df.production_companies.apply(extractProduction)

In [17]:
df.production_companies.iloc[0]

''

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

now inorder to apply countvectorizer to the whole dataset we need to 
combine all the values into one single sentence and make a single column

In [19]:
def combineRows(value):
    v = ' '.join(map(str,list(value.values)))
    
    return v
df1['text']=df1.apply(combineRows,axis=1)

<ipython-input-19-30709dab9bb0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['text']=df1.apply(combineRows,axis=1)


In [20]:
df1['text'].head()

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
Name: text, dtype: object

In [21]:
df1.head()

,keywords,cast,genres,director,production_companies,text
0,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,Action Adventure Fantasy Science Fiction,James Cameron,Ingenious Film Partners Twentieth Century Fox ...,culture clash future space war space colony so...
1,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Adventure Fantasy Action,Gore Verbinski,Walt Disney Pictures Jerry Bruckheimer Films S...,ocean drug abuse exotic island east india trad...
2,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Action Adventure Crime,Sam Mendes,Columbia Pictures Danjaq B24,spy based on novel secret agent sequel mi6 Dan...
3,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Action Crime Drama Thriller,Christopher Nolan,Legendary Pictures Warner Bros. DC Entertainme...,dc comics crime fighter terrorist secret ident...
4,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Action Adventure Science Fiction,Andrew Stanton,Walt Disney Pictures,based on novel mars medallion space travel pri...


combining original dataset with the "text" column of the new dataset

In [22]:
df['combined_features']=df1['text']

In [23]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,combined_features
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron,culture clash future space war space colony so...
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski,ocean drug abuse exotic island east india trad...
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes,spy based on novel secret agent sequel mi6 Dan...
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan,dc comics crime fighter terrorist secret ident...
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton,based on novel mars medallion space travel pri...


oops we got some Nan values in keywords this will create problems
so we need to get rid of those values

In [24]:
 df.keywords.isna()

0       False
1       False
2       False
3       False
4       False
        ...  
4798    False
4799     True
4800    False
4801     True
4802    False
Name: keywords, Length: 4803, dtype: bool

In [25]:
df.keywords.fillna(' ',inplace=True)

In [26]:
df.keywords.isna()

0       False
1       False
2       False
3       False
4       False
        ...  
4798    False
4799    False
4800    False
4801    False
4802    False
Name: keywords, Length: 4803, dtype: bool

In [27]:
#function to change every string value to lowercase
def toLowerDf(df):
    columns = list(df.columns)
    for col in columns:
        if df[col].dtype=='O':
            df[col]=df[col].str.lower()
    

In [28]:
toLowerDf(df)

In [29]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,combined_features
0,0,237000000,action adventure fantasy science fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,avatar,"in the 22nd century, a paraplegic marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""english""}, {""iso...",released,enter the world of pandora.,avatar,7.2,11800,sam worthington zoe saldana sigourney weaver s...,"[{'name': 'stephen e. rivkin', 'gender': 0, 'd...",james cameron,culture clash future space war space colony so...
1,1,300000000,adventure fantasy action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""english""}]",released,"at the end of the world, the adventure begins.",pirates of the caribbean: at world's end,6.9,4500,johnny depp orlando bloom keira knightley stel...,"[{'name': 'dariusz wolski', 'gender': 2, 'depa...",gore verbinski,ocean drug abuse exotic island east india trad...
2,2,245000000,action adventure crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,spectre,a cryptic message from bond’s past sends him o...,107.376788,...,"[{""iso_639_1"": ""fr"", ""name"": ""fran\u00e7ais""},...",released,a plan no one escapes,spectre,6.3,4466,daniel craig christoph waltz l\u00e9a seydoux ...,"[{'name': 'thomas newman', 'gender': 2, 'depar...",sam mendes,spy based on novel secret agent sequel mi6 dan...
3,3,250000000,action crime drama thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,the dark knight rises,following the death of district attorney harve...,112.312950,...,"[{""iso_639_1"": ""en"", ""name"": ""english""}]",released,the legend ends,the dark knight rises,7.6,9106,christian bale michael caine gary oldman anne ...,"[{'name': 'hans zimmer', 'gender': 2, 'departm...",christopher nolan,dc comics crime fighter terrorist secret ident...
4,4,260000000,action adventure science fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,john carter,"john carter is a war-weary, former military ca...",43.926995,...,"[{""iso_639_1"": ""en"", ""name"": ""english""}]",released,"lost in our world, found in another.",john carter,6.1,2124,taylor kitsch lynn collins samantha morton wil...,"[{'name': 'andrew stanton', 'gender': 2, 'depa...",andrew stanton,based on novel mars medallion space travel pri...


In [30]:
cv = CountVectorizer() 

count_matrix = cv.fit_transform(df['combined_features'])

similarity_scores = cosine_similarity(count_matrix)

In [31]:
similarity_scores

array([[1.        , 0.07325794, 0.08804509, ..., 0.0547791 , 0.        ,
        0.05763904],
       [0.07325794, 1.        , 0.09245003, ..., 0.05751973, 0.        ,
        0.03026138],
       [0.08804509, 0.09245003, 1.        , ..., 0.03456506, 0.09072184,
        0.        ],
       ...,
       [0.0547791 , 0.05751973, 0.03456506, ..., 1.        , 0.        ,
        0.06788442],
       [0.        , 0.        , 0.09072184, ..., 0.        , 1.        ,
        0.        ],
       [0.05763904, 0.03026138, 0.        , ..., 0.06788442, 0.        ,
        1.        ]])

In [32]:
#getting the index of the input movie given by the user
def get_index_movie_title(movie):
    movie = movie.lower()
    indexL = df.index[df['title']==movie].tolist()
    if len(indexL)==0:
        return -1
    else:
        return indexL[0]
    
def get_movie_title_from_index(index):
    return df.iloc[index]['title']
        

In [33]:
#getting the similar movies based on similarity scores
def get_similar_movies(movie_name,no_of_results=5):
    
    movie_index = get_index_movie_title(movie_name)
    if movie_index==-1:
        return "Sorry no movie named "+movie_name+" present in the database. Or please check the spelling"
    similar_movies = list(enumerate(similarity_scores[movie_index]))
    similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)
    similar_movies = similar_movies[1:no_of_results]
    results=[]
    
    for res in similar_movies:
        #here res is a tuple -> (index,similarity_score)
        value = get_movie_title_from_index(res[0])
        results.append(value)
        
    return results
        


In [34]:
get_similar_movies('batman begins',10)

['the dark knight rises',
 'the dark knight',
 'suicide squad',
 'batman: the dark knight returns, part 2',
 'batman returns',
 'watchmen',
 'batman & robin',
 'batman v superman: dawn of justice',
 'batman']

## NOTE : I found that not including the production company names works much better so inorder to do so you can remove production_companies from feature list in this notebook and try. The script version will not include the production company names.